In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!cp /content/drive/MyDrive/ML/data_utils.py /content
!cp /content/drive/MyDrive/ML/download.py /content

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import data_utils
import download
from tqdm import tqdm

def download_data():
    url = "https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz"
    download_dir = "./data"
    tqdm(download.maybe_download_and_extract(url,download_dir), desc='CIFAR10 Downloading')

In [ ]:
  # Class to initialize and apply K-nearest neighbour classfier
  class KNearestNeighbor(object):
      def __init__(self):
          pass

    # Method to initialize classifier with training data
    def train(self, X, y):
        self.X_train = X
        self.y_train = y

    # Method to predict labels 'compute_distances'
    def predict(self, X, k=1, num_loops=0):
        if num_loops == 0:
            dists = self.compute_distances(X)
        else:
            raise ValueError('Invalid value %d for num_loops' % num_loops)
        return self.predict_labels(dists, k=k)

    # Manhattan (L1) distances
    def compute_distances(self, X_test):
        num_test = X_test.shape[0]
        num_train = self.X_train.shape[0]
        dists = np.zeros((num_test, num_train))
        for i in range(num_test):
            for j in range(num_train):
                dists[i, j] = np.sum(np.abs(X_test[i, :] - self.X_train[j, :]))
        return dists


    # Method to predict labels
    def predict_labels(self, dists, k=1):
        num_test = dists.shape[0]
        y_pred = np.zeros(num_test)
        for i in range(num_test):
            closest_y_indices = np.argsort(dists[i])[:k]
            closest_y = self.y_train[closest_y_indices]
            y_pred[i] = np.argmax(np.bincount(closest_y))
        return y_pred

  def visualize_data(X_train, y_train):
      classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
      num_classes = len(classes)
      samples_per_class = 7
      for y, cls in enumerate(classes):
          idxs = np.flatnonzero(y_train == y)
          idxs = np.random.choice(idxs, samples_per_class, replace=False)
          for i, idx in enumerate(idxs):
              plt_idx = i * num_classes + y + 1
              plt.subplot(samples_per_class, num_classes, plt_idx)
              plt.imshow(X_train[idx].astype('uint8'))
              plt.axis('off')
              if i == 0:
                  plt.title(cls)
      plt.show()

In [ ]:
    if __name__ == "__main__":

    # Downloading CIFAR10 data
    download_data()
    cifar10_dir = r'./data/cifar-10-batches-py'
    X_train, y_train, X_test, y_test = data_utils.load_CIFAR10(cifar10_dir)
    print('Training data shape: ', X_train.shape)
    print('Training labels shape: ', y_train.shape)
    print('Test data shape: ', X_test.shape)
    print('Test labels shape: ', y_test.shape)

    # subsampling data 10000 training examples and 1000 test examples.
    num_training = 10000
    mask = list(range(num_training))
    X_train = X_train[mask]
    y_train = y_train[mask]

    num_test = 1000
    mask = list(range(num_test))
    X_test = X_test[mask]
    y_test = y_test[mask]

    # Reshape data
    X_train = X_train.reshape(X_train.shape[0], -1)
    X_test = X_test.reshape(X_test.shape[0], -1)

    print(X_train.shape, X_test.shape)


    # Initializing the classifier with training data
    classifier = KNearestNeighbor()
    classifier.train(X_train, y_train)
    dists = classifier.compute_distances(X_test)
    y_test_pred = classifier.predict_labels(dists, k=5)

    # accuracy
    num_correct = np.sum(y_test_pred == y_test)
    num_test = y_test.shape[0]
    accuracy = float(num_correct) / num_test
    print('Got %d / %d correct with k=5 => accuracy: %f' % (num_correct, num_test, accuracy))

    # 5-fold cross-validation
    num_folds = 5
    k_choices = [1, 3, 5, 8, 10, 12, 15, 20, 50, 100]

    X_train_folds = np.array_split(X_train, num_folds)
    y_train_folds = np.array_split(y_train, num_folds)
    k_to_accuracies = {}

    for k in tqdm(k_choices, desc='k choices'):
        k_to_accuracies[k] = []
        for fold in tqdm(range(num_folds)):
            X_train_cv = np.concatenate([X_train_folds[i] for i in range(num_folds) if i != fold])
            y_train_cv = np.concatenate([y_train_folds[i] for i in range(num_folds) if i != fold])
            X_valid_cv = X_train_folds[fold]
            y_valid_cv = y_train_folds[fold]

            classifier.train(X_train_cv, y_train_cv)
            dists_cv = classifier.compute_distances(X_valid_cv)
            y_valid_pred = classifier.predict_labels(dists_cv, k)

            # Compute accuracy
            num_correct_cv = np.sum(y_valid_pred == y_valid_cv)
            accuracy_cv = float(num_correct_cv) / len(y_valid_cv)
            k_to_accuracies[k].append(accuracy_cv)

    # Print computed accuracies
    for k in sorted(k_to_accuracies):
        for accuracy in k_to_accuracies[k]:
            print('k = %d, accuracy = %f' % (k, accuracy))


    for k in k_choices:
        k_to_accuracies[k] = []
        for num_knn in range(num_folds):

            X_train_folds_temp = X_train_folds[:]
            X_valid = X_train_folds_temp.pop(num_knn)
            X_train_temp = np.concatenate(X_train_folds_temp)
            y_train_folds_temp = y_train_folds[:]
            y_valid = y_train_folds_temp.pop(num_knn)
            y_train_temp = np.concatenate(y_train_folds_temp)

            classifier.train(X_train_temp, y_train_temp)
            dists = classifier.compute_distances(X_valid)
            y_test_pred = classifier.predict_labels(dists, k=k)

            # Compute accuracy
            num_correct = np.sum(y_test_pred == y_valid)
            num_test = y_valid.shape[0]
            accuracy = float(num_correct) / num_test
            k_to_accuracies[k].append(accuracy)

    print("Printing our 5-fold accuracies for varying values of k:")
    print()
    for k in sorted(k_to_accuracies):
        for accuracy in k_to_accuracies[k]:
            print('k = %d, accuracy = %f' % (k, accuracy))

    # average accuracy
    for k in sorted(k_to_accuracies):
        print('k = %d, avg. accuracy = %f' % (k, sum(k_to_accuracies[k])/num_folds))

    # Visualizing accuracies
    plt.figure(figsize=(10, 6))
    for k in k_choices:
        accuracies = k_to_accuracies[k]
        plt.scatter([k] * len(accuracies), accuracies)

    plt.title('Cross-validation on k')
    plt.xlabel('k value')
    plt.ylabel('Validation accuracy')
    plt.plot()

    accuracies_mean = np.array([np.mean(v) for k,v in sorted(k_to_accuracies.items())])
    accuracies_std = np.array([np.std(v) for k,v in sorted(k_to_accuracies.items())])
    best_k = k_choices[np.argmax(accuracies_mean)]
    print("Best k value:", best_k)


    classifier.train(X_train, y_train)
    dists = classifier.compute_distances(X_test)
    y_test_pred = classifier.predict_labels(dists, k=best_k)

    num_correct = np.sum(y_test_pred == y_test)
    accuracy = float(num_correct) / num_test
    print('Got %d / %d correct on test data => accuracy: %f' % (num_correct, num_test, accuracy))

- Download progress: 100.0%
Download finished. Extracting files.
Done.


CIFAR10 Downloading: 0it [00:00, ?it/s]


Training data shape:  (50000, 32, 32, 3)
Training labels shape:  (50000,)
Test data shape:  (10000, 32, 32, 3)
Test labels shape:  (10000,)
(10000, 3072) (1000, 3072)
Got 310 / 1000 correct with k=5 => accuracy: 0.310000


k choices: 100%|██████████| 10/10 [3:04:51<00:00, 1109.19s/it]


k = 1, accuracy = 0.320000
k = 1, accuracy = 0.322000
k = 1, accuracy = 0.297000
k = 1, accuracy = 0.293000
k = 1, accuracy = 0.297000
k = 3, accuracy = 0.315500
k = 3, accuracy = 0.295500
k = 3, accuracy = 0.300000
k = 3, accuracy = 0.290000
k = 3, accuracy = 0.286500
k = 5, accuracy = 0.324500
k = 5, accuracy = 0.317000
k = 5, accuracy = 0.323500
k = 5, accuracy = 0.290000
k = 5, accuracy = 0.301000
k = 8, accuracy = 0.324000
k = 8, accuracy = 0.326000
k = 8, accuracy = 0.321500
k = 8, accuracy = 0.300500
k = 8, accuracy = 0.314000
k = 10, accuracy = 0.329500
k = 10, accuracy = 0.323000
k = 10, accuracy = 0.325500
k = 10, accuracy = 0.291000
k = 10, accuracy = 0.313000
k = 12, accuracy = 0.327000
k = 12, accuracy = 0.316500
k = 12, accuracy = 0.324000
k = 12, accuracy = 0.289000
k = 12, accuracy = 0.307000
k = 15, accuracy = 0.323500
k = 15, accuracy = 0.331000
k = 15, accuracy = 0.317500
k = 15, accuracy = 0.294500
k = 15, accuracy = 0.311000
k = 20, accuracy = 0.319000
k = 20, accu

KeyboardInterrupt: 